In [8]:
%%capture
import delfi.distribution as dd
import delfi.utils.io as io
import numpy as np
import pickle

import delfi.inference as infer
import delfi.generator as dg
from delfi.simulator import GaussMixture
import delfi.summarystats as ds

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from parameters import ParameterSet
%run -i ../../DDELFI.py

In [9]:
output_dir = 'figs/'

In [10]:
params = ParameterSet({})

seed = 42

params.res = ParameterSet({})
params.res.reinit_weights = True
params.res.n_components = 2
params.res.n_hiddens = [20]
params.res.svi = False

params.run = ParameterSet({})
params.run.n_train = 1000
params.run.minibatch = 100
params.n_gradients = 100000

params.model = ParameterSet({})

In [11]:
params.model.prior_bounds = (-10, 10)

output_prefix = 'bimodal_gaussian'

def create_model(seed, params):
    m = GaussMixture(dim=1, bimodal=True, noise_cov=[1.0, 1.0], return_abs=True, seed=seed)
    p = dd.Uniform(lower=-10, upper=10, seed=seed)
    s = ds.Identity()
    g = dg.Default(model=m, prior=p, summary=s, seed=seed)
    
    obs = np.array([[5.]])
    
    p_true = dd.MoG(a=[0.5, 0.5], ms=[np.asarray([5.]), np.asarray([-5.])], Ss=[1.0*np.eye(1), 1.0*np.eye(1)], seed=seed)

    return g, obs, p_true

In [12]:
def run_algo(algo, proposal, seed, params):
    g, obs, _ = create_model(seed, params)
    
    res_kwargs = {'generator': g, 
                  'reg_lambda': 0.01,
                  'verbose' : True,
                  'obs': obs.copy(),
                  'seed': seed}

    res_kwargs.update(params.res)
    
    cbk_feature_layer=0
    
    round_cl=999
    minibatch=100
    epochs = [params.n_gradients // (params.run.n_train//minibatch)]
    
    optim_data = None
    
    if algo == 'CDELFI':
        inf = infer.CDELFI(**res_kwargs)
        log, train_data, posteriors = inf.run(n_rounds=1,
                                              impute_proposal=proposal,
                                              epochs=epochs,
                                              **params.run)
    elif algo == 'SNPE':
        inf = infer.SNPE(**res_kwargs)
        log, train_data, posteriors = inf.run(n_rounds=1,
                                              impute_proposal=proposal,
                                              epochs=epochs,
                                              **params.run)     
    elif algo == 'kSNPE':
        inf = infer.SNPE(**res_kwargs)
        log, train_data, posteriors = inf.run(n_rounds=1,
                                              impute_proposal=proposal,
                                              epochs=epochs,
                                              kernel_loss='x_kl', 
                                              cbk_feature_layer=cbk_feature_layer,
                                              **params.run)
    elif algo == 'DDELFI':
        inf = DDELFI(**res_kwargs)
        log, train_data, posteriors, optim_data = inf.run(n_rounds=1,
                                                          impute_proposal=proposal,
                                                          epochs=epochs,
                                                          **params.run)   
    else:
        raise ValueError("Unknown algorithm '{}'".format(algo))
    
    return {'algo': algo,
            'seed': seed,
            'generator': g,
            'log': log,
            'obs' : obs, 
            'impute_proposal' : proposal,
            'train_data': train_data,
            'posteriors': posteriors,
            'network' : inf.network,
            'stats_std' : inf.stats_std,
            'stats_mean' : inf.stats_mean,
            'optim_data' : optim_data,
            'cbk_feature_layer' : cbk_feature_layer
           }

In [23]:
#T_dofs = [ 1, 2, 3, 5, 10 ]
#alphas = [ 0.01, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0 ]

T_dofs = []
alphas = [ 0.05, 0.1, 0.2, 0.3, 0.5 ]

def create_proposals(seed, params):
    g, _, p_true = create_model(seed, params)
    prior = g.prior
    
    prop_Ts = { "Student's T (df = {})".format(dofs) : p_true.convert_to_T(dofs) for dofs in T_dofs }
    prop_alphas = { "Defensive (a = {})".format(alpha) : DefensiveDistribution(p_true, prior, alpha, seed=seed) for alpha in alphas }
    
    ret = { "Prior" : prior, "Posterior" : p_true }
    ret.update(prop_Ts)
    ret.update(prop_alphas)
    
    ret = { k: CroppedDistribution(ret[k], prior) for k in ret.keys() }
    return { "Prior" : ret["Prior"]}

In [24]:
proposals = create_proposals(seed, params)
algos = [ 'DDELFI' ]
seeds = range(98, 99)

outs = { algo : { proposal : [] for proposal in proposals } for algo in algos }

for seed in seeds:
    for algo in algos:
        for proposal_name in proposals:
            print("Seed: {}".format(seed))
            print("Algorithm: {}".format(algo))
            print("Proposal Prior: {}".format(proposal_name))
            proposal = proposals[proposal_name]
            out = run_algo(algo, proposal, seed, params)
            outs[algo][proposal_name].append(out)

Seed: 98
Algorithm: DDELFI
Proposal Prior: Prior


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

loss=[ 0.05238771]: : 0it [00:00, ?it/s]


> /Users/kaan/lfi-experiments/dist_cdelfi/mogtrain.py(299)train()
-> loss.backward()
(Pdb) p step
0
(Pdb) p self.get_mog()
*** AssertionError: p must sum to 1
(Pdb) q


BdbQuit: 

In [ ]:
_, _, p_true = create_model(seed=None, params=params)
COL = { 'DDELFI' : 'orange', 'SNPE' : 'green', 'kSNPE' : 'blue' }

pdf_filename = "{}/{}.pdf".format(output_dir, output_prefix)

with PdfPages(pdf_filename) as pdf:
    for proposal_name in proposals:
        proposal = proposals[proposal_name]
        plt.figure(figsize=(5 * len(algos),4))

        title = "Proposal prior: {}".format(proposal_name)
        xlist = np.linspace(*params.model.prior_bounds, 200).reshape(-1,1)
        plt.suptitle(title)

        for i, algo in enumerate(algos):
            ax = plt.subplot(1, len(algos), i+1)

            ax.set_title(algo)

            label = algo
            for out in outs[algo][proposal_name]:
                posterior = out['posteriors'][-1]

                plt.plot(xlist.reshape(-1), posterior.eval(xlist, log=False), COL[algo], alpha=0.5, linewidth=2, label=label)
                label = None

            plt.plot(xlist.reshape(-1), p_true.eval(xlist, log=False), 'k', linewidth=2, label="GT")
            plt.plot(xlist.reshape(-1), proposal.eval(xlist, log=False), 'r', ls='--', alpha=0.6, linewidth=2, label="Proposal")
            
            plt.xlim((params.model.prior_bounds))
            plt.yticks([])

            plt.legend()
            plt.tick_params(top='off', left='off', right='off', labelleft='off', labelbottom='on')

        pdf.savefig()
        plt.show()